In [ ]:
import os
import requests
from dotenv import load_dotenv
import time
import folium
import geopandas as gpd
from shapely.geometry import Polygon, shape

load_dotenv()

token = os.getenv("YANDEX_OAUTH_TOKEN")

# Заголовки для запроса к API
headers = {
    "Authorization": f"OAuth {token}"
}


In [2]:
# Тест получения сегмента
url = "https://api-audience.yandex.ru/v1/management/segments"

response = requests.get(url, headers=headers)
data = response.json()

# Находим сегмент с id = 48004577
segment = next((s for s in data['segments'] if s['id'] == 48004577), None)

if segment:
    print("Название:", segment['name'])
    print("Форма:", segment['form'])
    print("Охват:", segment.get('cookies_matched_quantity'))
    print("Координаты:", segment.get('polygons', segment.get('points')))
else:
    print("Сегмент не найден")

Название: 22
Форма: polygon
Охват: 6045
Координаты: [{'points': [{'latitude': 55.760204682312796, 'longitude': 49.156801915771425}, {'latitude': 55.757881588652154, 'longitude': 49.159891820556595}, {'latitude': 55.75933353847612, 'longitude': 49.16384003222652}, {'latitude': 55.76160815072086, 'longitude': 49.1605784660644}, {'latitude': 55.76088222511896, 'longitude': 49.159119344360306}, {'latitude': 55.760204682312796, 'longitude': 49.156801915771425}]}]


In [ ]:
headers = {
    "Authorization": f"OAuth {token}",
    "Content-Type": "application/json"
}

# ------------------------------
# Параметры сетки в Казани
# ------------------------------
base_lat, base_lon = 55.79, 49.12  # центр сетки
delta = 0.002  # шаг ~200 м между сегментами
rows, cols = 2, 5  # сетка 2x5 = 10 сегментов

created_segments = []  # для хранения id и полигонов

------------------------------
Создание 10 сегментов
------------------------------
for i in range(rows):
    for j in range(cols):
        # смещение для текущего квадрата
        lat_offset = i * delta
        lon_offset = j * delta

        # точки квадрата (замыкаем первый пункт)
        polygon = [
            {"latitude": base_lat + lat_offset, "longitude": base_lon + lon_offset},
            {"latitude": base_lat + lat_offset + delta, "longitude": base_lon + lon_offset},
            {"latitude": base_lat + lat_offset + delta, "longitude": base_lon + lon_offset + delta},
            {"latitude": base_lat + lat_offset, "longitude": base_lon + lon_offset + delta},
            {"latitude": base_lat + lat_offset, "longitude": base_lon + lon_offset}
        ]

        payload = {
            "segment": {
                "name": f"Test Polygon {i*cols + j + 1}",
                "geo_segment_type": "regular",
                "polygons": [{"points": polygon}]
            }
        }

        url = "https://api-audience.yandex.ru/v1/management/segments/create_geo_polygon"
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()
        print(f"Сегмент {i*cols + j + 1} статус:", response.status_code, data)

        if response.status_code == 200:
            created_segments.append(data['segment'])

        time.sleep(1)  # пауза, чтобы API не ругался



In [5]:

# ------------------------------
# Визуализация сетки на карте
# ------------------------------

created_segments = []

# Параметры сетки
base_lat, base_lon = 55.79, 49.12
delta = 0.003  # ~300 м
rows, cols = 20, 50  # сетка 2x5 = 10 сегментов

for i in range(rows):
    for j in range(cols):
        lat_offset = i * delta
        lon_offset = j * delta
        polygon = [
            {"latitude": base_lat + lat_offset, "longitude": base_lon + lon_offset},
            {"latitude": base_lat + lat_offset + delta, "longitude": base_lon + lon_offset},
            {"latitude": base_lat + lat_offset + delta, "longitude": base_lon + lon_offset + delta},
            {"latitude": base_lat + lat_offset, "longitude": base_lon + lon_offset + delta},
            {"latitude": base_lat + lat_offset, "longitude": base_lon + lon_offset}  # замыкаем полигон
        ]
        created_segments.append({
            "name": f"Test Polygon {i*cols + j + 1}",
            "polygons": [{"points": polygon}]
        })


m = folium.Map(location=[base_lat + delta, base_lon + delta], zoom_start=15)

for seg in created_segments:
    for poly in seg['polygons']:
        coords = [(p['latitude'], p['longitude']) for p in poly['points']]
        folium.Polygon(
            locations=coords,
            color='blue',
            weight=2,
            fill=True,
            fill_opacity=0.3,
            popup=seg['name']
        ).add_to(m)

m  # показываем карту в Jupyter
m.save("kazan_grid.html")